# ParIce + RMH + EN mono - map/pre-process
Í þessu reikniriti forvinnum við öll gögnin.

In [10]:
# The location of en-mono to read
en_mono_dir = '/work/haukurpj/data/filtered/en-monolingual'

# The location of where to write the results
en_mono_target_dir = '/work/haukurpj/data/mapped/en-monolingual'

# The location of rmh to read
is_mono_dir = '/work/haukurpj/data/filtered/risamalheild'

# The location of where to write the results
is_mono_target_dir = '/work/haukurpj/data/mapped/risamalheild'

# The location of parice to read
parice_dir = '/work/haukurpj/data/filtered_dedup/Parice1.0'

# The location of where to write the results
parice_target_dir = '/work/haukurpj/data/mapped_dedup/Parice1.0'

THREADS = 6

In [11]:
import pathlib

en_mono_dir = pathlib.Path(en_mono_dir)
en_mono_target_dir = pathlib.Path(en_mono_target_dir)
is_mono_dir = pathlib.Path(is_mono_dir)
is_mono_target_dir = pathlib.Path(is_mono_target_dir)
parice_dir = pathlib.Path(parice_dir)
parice_target_dir = pathlib.Path(parice_target_dir)
assert en_mono_dir.exists()
assert is_mono_dir.exists()
assert parice_dir.exists()
if not en_mono_target_dir.exists():
    en_mono_target_dir.mkdir()
if not is_mono_target_dir.exists():
    is_mono_target_dir.mkdir()
if not parice_target_dir.parent.exists():
    parice_target_dir.parent.mkdir()
if not parice_target_dir.exists():
    parice_target_dir.mkdir()

In [14]:
from frontend import bulk as b
from frontend import core as c
from frontend import definitions as d
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import re

URI = re.compile(r"((http(s)?:\/\/)|(www)|([-a-zA-Z0-9:%_\+.~#?&/=]+?@))+([-a-zA-Z0-9@:%_\+.~#?&/=]+)", re.IGNORECASE)
URI_SIMPLE = re.compile(r"([-a-zA-Z0-9@:%_\+.~#?&/=]+?)(\.is|\.com)", re.IGNORECASE)

reg = c.regexp
tok = c.tokenize
low = c.lowercase_normalize
second_reg = [
        d.SUB_PIPE,
        d.SUB_LT,
        d.SUB_GT,
        d.SUB_BRACKET_OPEN,
        d.SUB_BRACKET_CLOSE,
        d.SUB_FIX_PLACEHOLDERS
    ]
first_reg = [
        {
            'pattern': URI,
            'repl': '_uri_'
        },
        {
            'pattern': URI_SIMPLE,
            'repl': '_uri_'
        },
        d.SUB_EMPTY_BRACKETS
    ]

def preprocess_sent(sent, lang, method):
    sent = reg(sent, first_reg)
    sent = tok(sent, lang, method=method)
    sent = low(sent)
    sent = reg(sent, second_reg)
    return sent + '\n'

def bulk_process_sent(p_in, p_out, function):
    lang = b._lang(p_in)
    if str(lang) == 'is':
        method = "shallow"
    else:
        method = "moses"
    with p_in.open() as f_in, p_out.open('w+') as f_out:
        with ProcessPoolExecutor(max_workers=THREADS) as executor:
            results = tqdm(executor.map(
                function,
                f_in,
                chunksize=1000))
            for result in results:
                f_out.write(result)

In [15]:
from functools import partial

#bulk_process_sent(en_mono_dir.joinpath('mono.en'), 
#                  en_mono_target_dir.joinpath('mono.en'),
#                  partial(preprocess_sent, method='moses', lang=c.Lang.EN))
#bulk_process_sent(is_mono_dir.joinpath('rmh.is'),
#                  is_mono_target_dir.joinpath('rmh.is'),
#                  partial(preprocess_sent, method='shallow', lang=c.Lang.IS))
bulk_process_sent(parice_dir.joinpath('train.is'),
                  parice_target_dir.joinpath('train.is'),
                  partial(preprocess_sent, method='shallow', lang=c.Lang.IS))
bulk_process_sent(parice_dir.joinpath('train.en'),
                  parice_target_dir.joinpath('train.en'),
                  partial(preprocess_sent, method='moses', lang=c.Lang.EN))
bulk_process_sent(parice_dir.joinpath('dev.is'),
                  parice_target_dir.joinpath('dev.is'),
                  partial(preprocess_sent, method='shallow', lang=c.Lang.IS))
bulk_process_sent(parice_dir.joinpath('dev.en'),
                  parice_target_dir.joinpath('dev.en'),
                  partial(preprocess_sent, method='moses', lang=c.Lang.EN))
bulk_process_sent(parice_dir.joinpath('test-ees.is'),
                  parice_target_dir.joinpath('test-ees.is'),
                  partial(preprocess_sent, method='shallow', lang=c.Lang.IS))
bulk_process_sent(parice_dir.joinpath('test-ees.en'),
                  parice_target_dir.joinpath('test-ees.en'),
                  partial(preprocess_sent, method='moses', lang=c.Lang.EN))
bulk_process_sent(parice_dir.joinpath('test-ema.is'),
                  parice_target_dir.joinpath('test-ema.is'),
                  partial(preprocess_sent, method='shallow', lang=c.Lang.IS))
bulk_process_sent(parice_dir.joinpath('test-ema.en'),
                  parice_target_dir.joinpath('test-ema.en'),
                  partial(preprocess_sent, method='moses', lang=c.Lang.EN))
bulk_process_sent(parice_dir.joinpath('test-opensubtitles.is'),
                  parice_target_dir.joinpath('test-opensubtitles.is'),
                  partial(preprocess_sent, method='shallow', lang=c.Lang.IS))
bulk_process_sent(parice_dir.joinpath('test-opensubtitles.en'),
                  parice_target_dir.joinpath('test-opensubtitles.en'),
                  partial(preprocess_sent, method='moses', lang=c.Lang.EN))


258001it [00:34, 7575.63it/s]

0it [00:00, ?it/s]

28913it [00:00, 289126.83it/s]

57001it [00:00, 256655.57it/s]

69174it [00:00, 48951.31it/s] 

78204it [00:01, 23629.90it/s]

84846it [00:02, 16704.02it/s]

89793it [00:02, 13558.34it/s]

93539it [00:03, 15131.38it/s]

96784it [00:03, 11736.39it/s]

100001it [00:03, 11283.02it/s]

102063it [00:04, 8677.82it/s] 

106001it [00:04, 11223.84it/s]

108198it [00:04, 13011.01it/s]

110357it [00:04, 11656.16it/s]

112150it [00:05, 8731.94it/s] 

116001it [00:05, 9783.49it/s]

117404it [00:05, 8848.50it/s]

118603it [00:05, 7010.37it/s]

122001it [00:05, 8394.09it/s]

123155it [00:06, 7889.67it/s]

125001it [00:06, 8339.01it/s]

127001it [00:06, 9446.86it/s]

128111it [00:06, 7349.96it/s]

130001it [00:06, 8176.22it/s]

131001it [00:07, 8468.49it/s]

134001it [00:07, 8309.96it/s]

136001it [00:07, 9388.52it/s]

139001it [00:07, 11047.62it/s]

140295it [00:08, 7189.95it/s] 

142001it [00:08, 8196.66it/s]

144001it [00:08, 9040.55it/s]

146001i

553001it [00:50, 11106.46it/s]

554556it [00:50, 10407.96it/s]

555917it [00:50, 9392.03it/s] 

557099it [00:51, 6647.95it/s]

560001it [00:51, 8349.77it/s]

561286it [00:51, 7784.51it/s]

562387it [00:51, 6798.93it/s]

563315it [00:51, 5958.42it/s]

567001it [00:52, 7483.09it/s]

568110it [00:52, 6502.45it/s]

569036it [00:52, 4545.12it/s]

573001it [00:52, 5980.00it/s]

574192it [00:52, 6456.29it/s]

575264it [00:53, 5100.35it/s]

577001it [00:53, 5738.29it/s]

580001it [00:53, 7471.17it/s]

582001it [00:53, 6959.77it/s]

584001it [00:54, 7107.52it/s]

586001it [00:54, 7388.46it/s]

589001it [00:54, 9059.64it/s]

590243it [00:54, 7942.97it/s]

593001it [00:55, 8363.97it/s]

595001it [00:55, 8854.52it/s]

596026it [00:55, 8660.68it/s]

598001it [00:55, 8758.65it/s]

599001it [00:55, 7800.04it/s]

602001it [00:56, 8713.57it/s]

603001it [00:56, 8538.88it/s]

605001it [00:56, 8389.21it/s]

606001it [00:56, 8035.62it/s]

609001it [00:56, 9632.37it/s]

610125it [00:56, 8248.30it/s]

61200

985001it [01:36, 7846.84it/s]

988001it [01:37, 8730.98it/s]

989594it [01:37, 10100.30it/s]

990741it [01:37, 9497.55it/s] 

991794it [01:37, 9749.35it/s]

992843it [01:37, 9468.43it/s]

994001it [01:37, 8841.34it/s]

996001it [01:38, 6919.70it/s]

1000001it [01:38, 8781.86it/s]

1001262it [01:38, 6667.65it/s]

1002273it [01:38, 5478.60it/s]

1007001it [01:39, 6250.66it/s]

1008001it [01:39, 5889.49it/s]

1013001it [01:40, 6631.46it/s]

1014001it [01:40, 6504.17it/s]

1018001it [01:40, 8369.90it/s]

1019288it [01:40, 6210.88it/s]

1020306it [01:40, 6797.68it/s]

1024001it [01:41, 8946.88it/s]

1025706it [01:41, 9957.98it/s]

1027306it [01:41, 9183.78it/s]

1031001it [01:41, 10866.46it/s]

1032518it [01:41, 9308.11it/s] 

1036001it [01:41, 10822.63it/s]

1037414it [01:42, 9678.80it/s] 

1038636it [01:42, 7004.01it/s]

1043001it [01:42, 8970.66it/s]

1044470it [01:42, 8584.84it/s]

1045733it [01:42, 8275.64it/s]

1049001it [01:43, 10309.72it/s]

1050505it [01:43, 10081.16it/s]

1051846i

1529678it [02:16, 25557.30it/s]

1533001it [02:16, 26311.92it/s]

1536001it [02:16, 25963.53it/s]

1539001it [02:17, 25639.12it/s]

1542001it [02:17, 25391.77it/s]

1547001it [02:17, 27377.99it/s]

1549790it [02:17, 26601.53it/s]

1553001it [02:17, 27017.80it/s]

1556001it [02:17, 26246.16it/s]

1560001it [02:17, 23989.44it/s]

1565001it [02:17, 28372.05it/s]

1568203it [02:18, 23412.34it/s]

1572001it [02:18, 24242.90it/s]

1577001it [02:18, 28157.40it/s]

1580255it [02:18, 22877.08it/s]

1584001it [02:18, 24831.24it/s]

1587001it [02:18, 24811.16it/s]

1591001it [02:19, 23174.24it/s]

1596001it [02:19, 26483.06it/s]

1599001it [02:19, 26741.64it/s]

1602001it [02:19, 26517.33it/s]

1604806it [02:19, 25771.04it/s]

1608001it [02:19, 25175.06it/s]

1611001it [02:19, 26360.16it/s]

1614001it [02:19, 25321.47it/s]

1617572it [02:19, 27742.68it/s]

1620457it [02:20, 24320.03it/s]

1624001it [02:20, 26703.43it/s]

1626850it [02:20, 24864.31it/s]

1630001it [02:20, 24847.43it/s]

1632591it 

2190001it [02:42, 25233.40it/s]

2192603it [02:42, 23976.54it/s]

2195069it [02:42, 23924.79it/s]

2197509it [02:43, 23088.84it/s]

2201001it [02:43, 24113.16it/s]

2203452it [02:43, 22405.65it/s]

2206001it [02:43, 22721.22it/s]

2209001it [02:43, 22414.49it/s]

2212001it [02:43, 22237.53it/s]

2215001it [02:43, 23065.99it/s]

2219001it [02:43, 23412.46it/s]

2224001it [02:44, 25322.76it/s]

2227001it [02:44, 23290.78it/s]

2231001it [02:44, 22784.95it/s]

2236001it [02:44, 26011.38it/s]

2238799it [02:44, 24725.26it/s]

2242001it [02:44, 25817.60it/s]

2244701it [02:44, 23249.99it/s]

2248959it [02:45, 26915.60it/s]

2251956it [02:45, 24285.56it/s]

2255001it [02:45, 21552.76it/s]

2260001it [02:45, 23443.97it/s]

2262563it [02:45, 20562.18it/s]

2267001it [02:45, 21503.91it/s]

2272001it [02:46, 25361.30it/s]

2274929it [02:46, 24501.75it/s]

2278001it [02:46, 25379.65it/s]

2280744it [02:46, 24182.29it/s]

2284001it [02:46, 24011.01it/s]

2286511it [02:46, 23323.19it/s]

2289442it 

381001it [00:42, 7171.74it/s]

383001it [00:42, 7592.58it/s]

384001it [00:43, 6546.16it/s]

386001it [00:43, 7112.55it/s]

389001it [00:43, 7744.13it/s]

390001it [00:43, 6779.97it/s]

392001it [00:44, 7327.92it/s]

395001it [00:44, 7638.61it/s]

397001it [00:44, 8160.76it/s]

398001it [00:44, 6941.01it/s]

400001it [00:45, 8191.61it/s]

401001it [00:45, 7820.01it/s]

404001it [00:45, 7038.76it/s]

406001it [00:45, 8664.67it/s]

407118it [00:45, 8617.70it/s]

410001it [00:46, 8422.53it/s]

411001it [00:46, 8579.74it/s]

412001it [00:46, 4045.63it/s]

417001it [00:47, 5398.06it/s]

418239it [00:47, 5211.59it/s]

421001it [00:47, 6109.09it/s]

423001it [00:47, 7560.53it/s]

424220it [00:48, 6032.10it/s]

426631it [00:48, 7776.69it/s]

428032it [00:48, 8247.86it/s]

429301it [00:48, 8180.08it/s]

430430it [00:48, 6113.85it/s]

433001it [00:48, 7791.09it/s]

434285it [00:49, 8289.90it/s]

435476it [00:49, 7072.26it/s]

436468it [00:49, 6305.81it/s]

438001it [00:49, 7134.50it/s]

441001it

829001it [01:37, 6891.55it/s]

831001it [01:37, 7293.39it/s]

833001it [01:37, 8806.95it/s]

834082it [01:37, 6568.79it/s]

835001it [01:37, 6913.92it/s]

836001it [01:38, 7256.62it/s]

837001it [01:38, 7146.59it/s]

838001it [01:38, 7638.07it/s]

840001it [01:38, 7989.08it/s]

842001it [01:38, 8940.65it/s]

844001it [01:39, 8255.69it/s]

845001it [01:39, 7689.62it/s]

846001it [01:39, 7630.31it/s]

849001it [01:39, 9509.05it/s]

850231it [01:39, 6892.83it/s]

851218it [01:39, 7224.12it/s]

852153it [01:39, 6905.29it/s]

853001it [01:40, 7292.90it/s]

855001it [01:40, 8702.61it/s]

856066it [01:40, 6283.52it/s]

858001it [01:40, 6810.41it/s]

860001it [01:41, 6155.18it/s]

862001it [01:41, 7262.31it/s]

864001it [01:41, 7690.11it/s]

866001it [01:41, 9204.27it/s]

867141it [01:41, 7415.71it/s]

868092it [01:41, 7464.32it/s]

870001it [01:42, 7244.64it/s]

871001it [01:42, 7001.70it/s]

873001it [01:42, 7862.64it/s]

874001it [01:42, 8268.93it/s]

876001it [01:42, 7786.32it/s]

877001it

1223001it [02:22, 9056.20it/s]

1225001it [02:22, 7927.70it/s]

1226001it [02:22, 8021.13it/s]

1229001it [02:23, 9068.24it/s]

1231001it [02:23, 8622.84it/s]

1233001it [02:23, 10027.39it/s]

1235001it [02:23, 9955.66it/s] 

1237001it [02:23, 8724.51it/s]

1241001it [02:24, 9676.20it/s]

1243001it [02:24, 8567.31it/s]

1246001it [02:24, 10694.02it/s]

1247428it [02:24, 8916.60it/s] 

1249001it [02:25, 7180.83it/s]

1252001it [02:25, 8856.53it/s]

1253272it [02:25, 9445.90it/s]

1255001it [02:25, 6540.10it/s]

1259001it [02:26, 8446.49it/s]

1261001it [02:26, 7565.82it/s]

1263001it [02:26, 8913.96it/s]

1265001it [02:26, 10251.39it/s]

1266390it [02:26, 10967.04it/s]

1267755it [02:27, 8783.52it/s] 

1269001it [02:27, 8988.37it/s]

1271001it [02:27, 9730.41it/s]

1272119it [02:27, 9205.40it/s]

1273147it [02:27, 7584.13it/s]

1275001it [02:27, 7966.25it/s]

1277001it [02:28, 8377.15it/s]

1279001it [02:28, 8538.50it/s]

1280001it [02:28, 7489.01it/s]

1281001it [02:28, 7714.14it/s]

1

1683520it [03:04, 10585.79it/s]

1687001it [03:05, 11121.02it/s]

1688194it [03:05, 10433.80it/s]

1690001it [03:05, 11424.68it/s]

1693001it [03:05, 11376.59it/s]

1694191it [03:05, 10715.96it/s]

1696001it [03:05, 12018.63it/s]

1699001it [03:06, 11780.17it/s]

1700238it [03:06, 11069.95it/s]

1701395it [03:06, 10605.68it/s]

1705001it [03:06, 11260.48it/s]

1706166it [03:06, 10432.38it/s]

1707249it [03:07, 10013.44it/s]

1711001it [03:07, 10894.51it/s]

1712135it [03:07, 10719.25it/s]

1713239it [03:07, 10305.11it/s]

1717001it [03:07, 11117.35it/s]

1719001it [03:08, 10138.22it/s]

1723001it [03:08, 11250.57it/s]

1724194it [03:08, 11414.69it/s]

1725384it [03:08, 10092.07it/s]

1729001it [03:08, 11022.55it/s]

1730172it [03:08, 11119.56it/s]

1731333it [03:09, 9693.53it/s] 

1735001it [03:09, 10864.75it/s]

1736181it [03:09, 8656.71it/s] 

1737183it [03:09, 7220.94it/s]

1741001it [03:09, 8641.85it/s]

1742081it [03:10, 8395.62it/s]

1743073it [03:10, 6941.18it/s]

1747001it [03:

2089001it [03:40, 11905.76it/s]

2090323it [03:40, 10495.19it/s]

2093001it [03:40, 10623.75it/s]

2095001it [03:41, 12072.02it/s]

2096336it [03:41, 10961.36it/s]

2099001it [03:41, 10988.73it/s]

2101001it [03:41, 12015.03it/s]

2102285it [03:41, 11270.47it/s]

2105001it [03:42, 10755.54it/s]

2107001it [03:42, 12080.86it/s]

2108299it [03:42, 10917.99it/s]

2111001it [03:42, 10699.91it/s]

2113001it [03:42, 11235.27it/s]

2114174it [03:42, 9911.32it/s] 

2117001it [03:43, 10732.30it/s]

2119001it [03:43, 10919.26it/s]

2120139it [03:43, 9941.19it/s] 

2123001it [03:43, 10581.77it/s]

2125001it [03:43, 11327.92it/s]

2126181it [03:43, 10553.44it/s]

2129001it [03:44, 10545.30it/s]

2131001it [03:44, 11552.64it/s]

2132207it [03:44, 10418.39it/s]

2135001it [03:44, 10805.58it/s]

2137001it [03:44, 11610.40it/s]

2138210it [03:44, 10966.21it/s]

2141001it [03:45, 11249.49it/s]

2143001it [03:45, 11726.28it/s]

2144200it [03:45, 11422.67it/s]

2147001it [03:45, 11170.21it/s]

2149001it 